In [2]:
import pandas as pd
import os
from sklearn.model_selection import KFold

In [10]:
dataset = "libraries"

benchmarks = {}
df = pd.read_csv(f"../results/non_verified_{dataset}.csv")

# remove duplicates
df.drop_duplicates(subset=["Original Method", "Assertion"], inplace=True)

k = len(df)

# rewrite the path of "New Method File" but keep the same file name
df["New Method File"] = df["New Method File"].apply(
    lambda x: os.path.join("/exp/dafny_repair/results/", os.path.basename(x))
)


kf = KFold(n_splits=k)

training_sets = []
testing_sets = []

for train_index, test_index in kf.split(df):
    training = df.iloc[train_index]
    testing = df.iloc[test_index]
    training_sets.append(training)
    testing_sets.append(testing)
    # training.to_csv(f"../results/training_{dataset}_k{k}_id{train_index}.csv", index=False)
    # testing.to_csv(f"../results/testing_{dataset}_k{k}_id{test_index}.csv", index=False)
if not os.path.exists(f"../results/tmp_{dataset}"):
    os.makedirs(f"../results/tmp_{dataset}")
training_files = []
testing_files = []
for i, (training, testing) in enumerate(zip(training_sets, testing_sets)):
    training_file = f"../results/tmp_{dataset}/training_{dataset}_k{k}_{i}.csv"
    testing_file = f"../results/tmp_{dataset}/testing_{dataset}_k{k}_{i}.csv"
    training.to_csv(training_file, index=False)
    testing.to_csv(testing_file, index=False)
    training_files.append(training_file)
    testing_files.append(testing_file)
benchmarks[dataset] = {
    "training": training_sets,
    "testing": testing_sets,
    "training_file": training_files,
    "testing_file": testing_files,
}
configs = ["configs/main/config_repos/config_llm_libraries_randomExample.yaml"]

for config in configs:
    file_path_without_ext = os.path.splitext(config)[0]
    type_config = file_path_without_ext.split("_")[-1]

    if not os.path.exists(f"./results_llm/tmp_{dataset}"):
        os.makedirs(f"./results_llm/tmp_{dataset}")
    for i, _ in enumerate(benchmarks[dataset]["training"]):
        output_file = (
            f"./results_llm/tmp_{dataset}/output_{type_config}_{dataset}_{i}.csv"
        )

        command = [
            "poetry",
            "run",
            "python",
            "src/test_harness.py",
            "llm",
            config,  # Replace with the name of the config
            "-p",
            benchmarks[dataset]["testing_file"][i][
                :
            ],  # Replace with the name of the bench
            "-o",
            output_file,  # Replace with the name of the output file
        ]
        command += ["-t", benchmarks[dataset]["training_file"][i][:]]
        print(" ".join(command))

poetry run python src/test_harness.py llm configs/main/config_repos/config_llm_libraries_randomExample.yaml -p ../results/tmp_libraries/testing_libraries_k58_0.csv -o ./results_llm/tmp_libraries/output_randomExample_libraries_0.csv -t ../results/tmp_libraries/training_libraries_k58_0.csv
poetry run python src/test_harness.py llm configs/main/config_repos/config_llm_libraries_randomExample.yaml -p ../results/tmp_libraries/testing_libraries_k58_1.csv -o ./results_llm/tmp_libraries/output_randomExample_libraries_1.csv -t ../results/tmp_libraries/training_libraries_k58_1.csv
poetry run python src/test_harness.py llm configs/main/config_repos/config_llm_libraries_randomExample.yaml -p ../results/tmp_libraries/testing_libraries_k58_2.csv -o ./results_llm/tmp_libraries/output_randomExample_libraries_2.csv -t ../results/tmp_libraries/training_libraries_k58_2.csv
poetry run python src/test_harness.py llm configs/main/config_repos/config_llm_libraries_randomExample.yaml -p ../results/tmp_librari

In [13]:
benchmarks["libraries"]["testing"][0]

,Index,Original File,Original Method,Original Method Time,Original Method Result,Original Method File,Assertion,Time Difference,New Method File,New Method,New Method Time,New Method Result,New Result File
0,30,/exp/test_projects/libraries/src/JSON/ZeroCopy...,MembersImpl,0.908584,Correct,./results/MembersImpl_0.txt,"assert wr == MembersSpec(obj, members[..0], wr...",NaN,/exp/dafny_repair/results/MembersImpl_fix_30.dfy,MembersImpl,1.000806,Errors,./results/MembersImpl_30.txt


In [14]:
benchmarks["libraries"]["testing"][1]

,Index,Original File,Original Method,Original Method Time,Original Method Result,Original Method File,Assertion,Time Difference,New Method File,New Method,New Method Time,New Method Result,New Result File
1,31,/exp/test_projects/libraries/src/JSON/ZeroCopy...,MembersImpl,0.908584,Correct,./results/MembersImpl_0.txt,assert members[..i+1][..i] == members[..i];\n,NaN,/exp/dafny_repair/results/MembersImpl_fix_31.dfy,MembersImpl,2.33283,Errors,./results/MembersImpl_31.txt
